# 휠 얼라이먼트

### 조향 값으로 이동하는 방향에 따라서 오일러 각을 받아 들여서 직진주행을 학습함

* ### 조향 모터의 오차 보정 인공지능 예제  

* ### 실제 차량의 휠 얼라인먼트를 보정하는 과정(핸들을 중앙에 놓았음에도 차량 진행 방향이 한쪽에 치우치는 현상)과 유사

In [2]:
from pop import Pilot
import numpy as np
import time

Car = Pilot.AutoCar()
Car.setSensorStatus(euler=1)

* ### 여러 각도로 조향하고, 9축 센서의 오일러 각 값 중 yaw 값을 읽어 저장

* ### 센서 측정 시간을 고려해 조향 후 0.5초가 지난 시점에 9축 센서 값 읽음  

* ### 다음 데이터 수집을 위해 전진한 만큼 후진하여 제자리로 복귀  

* ### 이 과정을 여러번 반복해 데이터셋을 많이 수집할수록 정확한 오차 보정 가능

In [2]:
dataset={ 'euler' : [], 'steer' : [] }

for n in np.arange(-0.9, 1.1, 0.3):
    n = round(n,1)
    
    Car.steering = n
    Car.forward(30)

    time.sleep(1)

    m = Car.getEuler('yaw')

    time.sleep(1)

    Car.backward(30)

    time.sleep(2)

    Car.stop()

    dataset['euler'].append(m)
    dataset['steer'].append(n)

    print({ 'euler' : m , 'steer' : n })

{'euler': 251.125, 'steer': -0.9}
{'euler': 250.75, 'steer': -0.6}
{'euler': 258.562, 'steer': -0.3}
{'euler': 259.875, 'steer': -0.0}
{'euler': 259.75, 'steer': 0.3}
{'euler': 260.562, 'steer': 0.6}
{'euler': 263.75, 'steer': 0.9}


* ### 오일러 각은 0~360 사이의 값으로 반환
    * #### 360도 혹은 0도 전후로 측정될 때가 있음
        * ##### 예를 들어, 조향 값이 -0.9일 때 350, 조향 값이 0.9일 때 10인 경우
* ### 원활한 모델 생성을 위해 값 수정

In [3]:
import numpy as np

max_value = np.array(dataset['euler']).max()
min_value = np.array(dataset['euler']).min()
if max_value - min_value > 300:
    for num, eu in enumerate(dataset['euler']):
        if eu > 180:
            dataset['euler'][num] = round(eu-360, 3)

* ### 선형 회귀를 위해 pop.AI 라이브러리 import   
* ### Linear_Regression 객체 생성    
    * #### X_data와 Y_data에 각각 euler와 steer를 지정   
* ### train() 메소드의 파라미터에 times를 5000으로 설정. 데이터 학습   
    * #### 충분한 학습을 위해 times 설정 

In [4]:
from pop import AI

LR=AI.Linear_Regression()
LR.X_data=dataset['euler']
LR.Y_data=dataset['steer']

LR.train(times=5000, print_every=100)

NameError: name 'dataset' is not defined

* ### 학습된 모델의 run() 메소드에 43을 입력하여 실행
    * #### 오일러 각의 yaw 값이 43에 가까울 때의 조향 값이 정중앙
        * ##### 해당 값은 데이터 수집 당시 차량의 방향에 따라 달라질 수 있음

In [12]:
value = LR.run([43])
print(value)

[[0.00059758]]


* ### 정중앙으로 주행하기 위한 조향 제어값을 알 수 있고, 이 값을 입력하여 조향 보정 

In [7]:
Car.steering = value[0][0]

In [8]:
Car.forward(40)
time.sleep(2)
Car.stop()

* ### 차량이 비교적 정확하게 직진하는 것을 확인 가능   
* ### 단, 베어링으로 인한 오차가 발생하여 완벽한 보정은 불가능  
* ### 모델은 학습시 자동저장, 아래는 모델을 불러와서 사용하는 법

In [5]:
LR.load(path="linear_regression_models")
value = LR.run([43])
print(value)

[[0.00059758]]
